In [1]:
import ktrain
from ktrain import text as txt
import pandas as pd

In [2]:
train_small = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/train_small.csv")
test_small = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")

In [3]:
test_small.shape

(200000, 5)

In [4]:
train_small.shape

(800000, 2)

In [5]:
small_x_train, small_y_train = train_small['text'].tolist(), train_small['stars'].tolist()

In [6]:
x_test, y_test = test_small['text'].tolist(), test_small['stars'].tolist()

In [7]:
import pickle

In [8]:
import boto3
from botocore.exceptions import ClientError

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [9]:
bucket = "yelp-dataset-pt-9"

In [15]:
trn, val, preproc = txt.texts_from_array(x_train=small_x_train, y_train=small_y_train,
                                          x_test=x_test, y_test=y_test,
                                          preprocess_mode='distilbert',
                                          maxlen=512)

task: text regression (supply class_names argument if this is supposed to be classification task)
preprocessing train...
language: en
train sequence lengths:
	mean : 109
	95percentile : 306
	99percentile : 520


preprocessing test...
language: en
test sequence lengths:
	mean : 109
	95percentile : 305
	99percentile : 522


In [16]:
large_or_small = "small"

In [17]:
pickle.dump( trn, open( f"{large_or_small}/trn.p", "wb" ), protocol=4 )
pickle.dump( val, open( f"{large_or_small}/val.p", "wb" ), protocol=4 )
pickle.dump( preproc, open( f"{large_or_small}/preproc.p", "wb" ), protocol=4 )

In [19]:
upload_file(f"{large_or_small}/preproc.p", bucket, f'spencer/data/sentiment/en/ktrain/regression/long/{large_or_small}/preproc.p')

True

In [20]:
upload_file(f"{large_or_small}/val.p", bucket, f'spencer/data/sentiment/en/ktrain/regression/long/{large_or_small}/val.p')

True

In [24]:
upload_file(f"{large_or_small}/trn.p", bucket, f'spencer/data/sentiment/en/ktrain/regression/long/{large_or_small}/trn.p')

True